# Dataclasses
 * [Source](https://www.youtube.com/watch?v=T-TwcmT6Rcw)
 * [Slides](https://www.dropbox.com/s/m8pwkkz43qz5pgt/HettingerPycon2018.pdf)

In [33]:
from dataclasses import dataclass, fields
from dataclasses import asdict, astuple, replace, field
from pprint import pprint as pp

## Creating Dataclass

In [5]:
@dataclass
class Color:
    hue: int
    saturation: float
    lightness: float = 0.5

## Creating object

In [6]:
c = Color(33, 1.)
c  # nice wrapper to display object, not available in normal classes

Color(hue=33, saturation=1.0, lightness=0.5)

In [7]:
print(c.hue, c.saturation, c.lightness)

33 1.0 0.5


## Replacing Attributes

In [8]:
# dataclasses are mutable by default, to make them immutable use @dataclass(frozen=True)
c.hue = 43
print(c)

Color(hue=43, saturation=1.0, lightness=0.5)


In [9]:
replace(c, hue=53)  # creates new obj
print(c)

Color(hue=43, saturation=1.0, lightness=0.5)


In [10]:
d = replace(c, hue=53)  # creates new obj
print(d)

Color(hue=53, saturation=1.0, lightness=0.5)


## Display attributes as dictionary/tuple

In [12]:
asdict(c)

{'hue': 43, 'saturation': 1.0, 'lightness': 0.5}

In [13]:
astuple(d)

(53, 1.0, 0.5)

## Automatic Annotations

In [15]:
Color.__annotations__

{'hue': int, 'saturation': float, 'lightness': float}

## Iterating over list of class objects

In [17]:
# ordering is not supported by default 
colors = [
    Color(33, 0.10), 
    Color(12, 0.17), 
    Color(34, 0.13), 
    Color(65, 0.19), 
]

pp(sorted(colors))

TypeError: '<' not supported between instances of 'Color' and 'Color'

In [18]:
@dataclass(order=True)
class Color:
    hue: int
    saturation: float
    lightness: float = 0.5

In [19]:
colors = [
    Color(33, 0.10), 
    Color(12, 0.17), 
    Color(34, 0.13), 
    Color(65, 0.19), 
]

pp(sorted(colors))

[Color(hue=12, saturation=0.17, lightness=0.5),
 Color(hue=33, saturation=0.1, lightness=0.5),
 Color(hue=34, saturation=0.13, lightness=0.5),
 Color(hue=65, saturation=0.19, lightness=0.5)]


## Additional Functionalities

In [23]:
from datetime import datetime

@dataclass(order=True, unsafe_hash=True)
class Employee:
    emp_id: int = field()
    name: str = field()
    gender: str = field()
    salary: int = field(hash=False, repr=False, metadata={"units":"bitcoins"})
    age: int = field(hash=False)
    viewed_by: list = field(default_factory=list, compare=False, repr=False)
    
    def access(self, viewer_id):
        self.viewed_by.append((viewer_id, datetime.now()))

In [25]:
e1 = Employee(
    emp_id = "1234567", # as str
    name = "Abc",
    gender = "male",
    salary = 500000, 
    age = 0x30, # in Hex
)

e2 = Employee(
    emp_id = "98765432", # as str
    name = "Xyz",
    gender = "female",
    salary = 900000, 
    age = 0x20, # in Hex
)

In [26]:
# doesn't display sensitive fields
print(e1)
print(e2)

Employee(emp_id='1234567', name='Abc', gender='male', age=48)
Employee(emp_id='98765432', name='Xyz', gender='female', age=32)


In [27]:
e1.access("me")
e1.access("you")

In [28]:
pp(e1.viewed_by) 

[('me', datetime.datetime(2020, 9, 16, 19, 23, 19, 963526)),
 ('you', datetime.datetime(2020, 9, 16, 19, 23, 19, 963526))]


In [29]:
pp(sorted([e1, e2]))  # not sorted on all fields

[Employee(emp_id='1234567', name='Abc', gender='male', age=48),
 Employee(emp_id='98765432', name='Xyz', gender='female', age=32)]


In [30]:
# employees can be hashed
assignments = {e1:"Data Scientist", e2:"SDE"}
pp(assignments)

{Employee(emp_id='1234567', name='Abc', gender='male', age=48): 'Data '
                                                                'Scientist',
 Employee(emp_id='98765432', name='Xyz', gender='female', age=32): 'SDE'}


In [36]:
pp(fields(e1)[3])

Field(name='salary',type=<class 'int'>,default=<dataclasses._MISSING_TYPE object at 0x000001B6690A0648>,default_factory=<dataclasses._MISSING_TYPE object at 0x000001B6690A0648>,init=True,repr=False,hash=False,compare=True,metadata=mappingproxy({'units': 'bitcoins'}),_field_type=_FIELD)
